In [1]:
# 불러오기
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
import requests
import os
import json
import numpy as np
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import re
import time
import datetime

In [2]:
# 이미지 다운로드하는 함수
def download_image(url, save_path):
    response = requests.get(url)
    response.raise_for_status()
    with open(save_path, 'wb') as file:
        file.write(response.content)

In [9]:
def get_img(goods_id,review_no,num):
    # 후기 개수
    count = len(soup.find_all("div",{"class":"review-list"})[num].find_all('img'))
    
    # 사진 카운트
    k = 0
    # 후기 사진만 저장하기
    for i in range(count):
        url = soup.find_all("div",{"class":"review-list"})[num].find_all('img')[i].get("src")
        if ".view" in url:
            file_url = 'https:' + re.sub(r'\.view$', '', url)
            # 사진 저장 경로
            save_path = f"review/{goods_id}_{review_no}_{k}.jpg"
            
            # 사진 다운로드 함수
            download_image(file_url, save_path)
            k += 1
            print(goods_id,review_no,num,count,k,file_url)

In [3]:
# page랑 brand 같이 url에 주면 그 페이지 review 크롤링
def get_review(url):
    # 제품 url 가져오기
    # BeautifulSoup
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,"html.parser")
    review = []
    review_count = len(soup.find_all("div",{"class":"review-list"}))
    for i in range(review_count):
        try:
            # goods_id
            url = soup.find_all("div",{"class":"review-list"})[i].find_all('a',{"class":"review-goods-information__link"})[0].get('href')
            match = re.search(r'goods/(\d+)/', url)
            if match:
                goods_id = match.group(1)

            # user_id
            user_id = soup.find_all("div",{"class":"review-list"})[i].find_all("p")[0].text

            # date
            date = soup.find_all("div",{"class":"review-list"})[i].find_all("p")[1].text

            # Size 
            size = re.sub(r'\s+', '', soup.find_all("div",{"class":"review-list"})[i].find_all("span",{"class":"review-goods-information__option"})[0].text)

            # text
            review_text = soup.find_all("div",{"class":"review-list"})[i].find_all("div",{"class":"review-contents__text"})[0].text

            # url
            goods_url = soup.find_all("div",{"class":"review-list"})[i].find_all('a')[2].get('href')
            #brand
            goods_brand = soup.find_all("div",{"class":"review-list"})[i].find_all('a')[3].text
            # goods name
            goods_name = soup.find_all("div",{"class":"review-list"})[i].find_all('a')[4].text

            # 도움돼요 /스타일 좋아요 / 평가
            help_eval = soup.find_all("div",{"class":"review-list"})[i].find_all('li',{"class":"review-evaluation-button__list"})[0].find('span').text
            style_eval = soup.find_all("div",{"class":"review-list"})[i].find_all('li',{"class":"review-evaluation-button__list"})[1].find('span').text
            eval_list = [item.get_text(strip=True) for item in soup.find_all("div",{"class":"review-list"})[i].find_all('li',{"class":"review-evaluation__item"})]
            rating_str = re.findall(r'\d+', soup.find_all("div",{"class":"review-list"})[i].find_all('span',{"class":"review-list__rating__active"})[0].get("style"))[0]
            rating = int(rating_str) / 20

            # review_no
            review_no = soup.find_all("div",{"class":"review-list"})[i].find_all("div",{"class":"review-contents"})[0].get("data-review-no")

            # 사진 크롤링
            img_count = len(soup.find_all("div",{"class":"review-list"})[i].find_all('img'))

            # 사진 카운트
            img_num = 0
            # 후기 사진만 저장하기
            for j in range(img_count):
                url = soup.find_all("div",{"class":"review-list"})[i].find_all('img')[j].get("src")
                if ".view" in url:
                    file_url = 'https:' + re.sub(r'\.view$', '', url)
                    # 사진 저장 경로
                    save_path = f"review/{goods_id}_{review_no}_{img_num}.jpg"

                    # 사진 다운로드 함수
                    download_image(file_url, save_path)
                    img_num += 1
#                     print(goods_id,review_no,j,img_count,img_num,file_url)

            # user 정보 ('여성 ', ' 160cm ', ' 52kg') / 등록되어 있지 않은 경우도 있음
            if soup.find_all("div",{"class":"review-list"})[i].find_all("p",{"class":"review-profile__body_information"}) != []:
                user_sex, user_height, user_weight = soup.find_all("div",{"class":"review-list"})[i].find_all("p",{"class":"review-profile__body_information"})[0].text.split("·")
                review.append([goods_id,user_id,date,size,review_text,goods_url,goods_brand,goods_name,help_eval,style_eval,eval_list,rating,review_no,user_sex, user_height, user_weight])
            else:
                review.append([goods_id,user_id,date,size,review_text,goods_url,goods_brand,goods_name,help_eval,style_eval,eval_list,rating,review_no])
        except:
            pass
    return review
    

In [5]:
path = r"C:\Users\NT550\Desktop\chromedriver.exe" # 크롬드라이버 경로 입력
s = Service(path)
chrome_options = Options()
chrome_options.add_argument('--headless')  # 화면에 브라우저가 나타나지 않도록 headless 모드로 실행합니다.

driver = webdriver.Chrome(service=s)

brand = pd.read_excel("brand_name_real.xlsx")
brand.drop(columns=['Unnamed: 0','Unnamed: 2'],inplace=True)
brand_list = [text.replace(' ', '').lower() for text in list(brand.loc[brand['나머지']==3,0])]

review = []
count = 0
fail_brand = []

for brand in brand_list:
    try:
        url = f'https://www.musinsa.com/goods/reviews/lists?type=style&searchYear=2022&searchMonth=&searchDay=&maxRt=2022&minRt=2009&brand={brand}&page=1&sort=new'
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,"html.parser")
        tot_page = int(re.sub(r'[^\d]', '', soup.find_all("span",{"class":"totalPagingNum"})[0].text))
        if tot_page >= 5000 : tot_page = 4999
        for page in range(1,tot_page+1):
            url = f'https://www.musinsa.com/goods/reviews/lists?type=style&searchYear=2022&searchMonth=&searchDay=&maxRt=2022&minRt=2009&brand={brand}&page={page}&sort=comment_new_desc'
            review = review + get_review(url)
            count += 1
            print("{0}의 {1} page 중에 {2} page까지 완료 / {3}".format(brand,tot_page,page,count))
            if count % 100 == 0:
                review_df = pd.DataFrame(review,columns = ["goods_id","user_id","date","size","text","goods_url","goods_brand","goods_name","help_eval","style_eval","eval_list","rating","review_no","user_sex", "user_height", "user_weight"])
                review_df.to_csv(f"review/review_{count*20}.csv")
                print(f"review 저장완료_{count*20}")
    except:
        fail_brand.append(brand)
        pass

suare의 1205 page 중에 1 page까지 완료 / 1
suare의 1205 page 중에 2 page까지 완료 / 2
suare의 1205 page 중에 3 page까지 완료 / 3
suare의 1205 page 중에 4 page까지 완료 / 4
suare의 1205 page 중에 5 page까지 완료 / 5
suare의 1205 page 중에 6 page까지 완료 / 6
suare의 1205 page 중에 7 page까지 완료 / 7
suare의 1205 page 중에 8 page까지 완료 / 8
suare의 1205 page 중에 9 page까지 완료 / 9
suare의 1205 page 중에 10 page까지 완료 / 10
suare의 1205 page 중에 11 page까지 완료 / 11
suare의 1205 page 중에 12 page까지 완료 / 12
suare의 1205 page 중에 13 page까지 완료 / 13
suare의 1205 page 중에 14 page까지 완료 / 14
suare의 1205 page 중에 15 page까지 완료 / 15
suare의 1205 page 중에 16 page까지 완료 / 16
suare의 1205 page 중에 17 page까지 완료 / 17
suare의 1205 page 중에 18 page까지 완료 / 18
suare의 1205 page 중에 19 page까지 완료 / 19
suare의 1205 page 중에 20 page까지 완료 / 20
suare의 1205 page 중에 21 page까지 완료 / 21
suare의 1205 page 중에 22 page까지 완료 / 22
suare의 1205 page 중에 23 page까지 완료 / 23
suare의 1205 page 중에 24 page까지 완료 / 24
suare의 1205 page 중에 25 page까지 완료 / 25
suare의 1205 page 중에 26 page까지 완료 / 26
suare의 1205 page 중에 27 page까지 

suare의 1205 page 중에 211 page까지 완료 / 211
suare의 1205 page 중에 212 page까지 완료 / 212
suare의 1205 page 중에 213 page까지 완료 / 213
suare의 1205 page 중에 214 page까지 완료 / 214
suare의 1205 page 중에 215 page까지 완료 / 215
suare의 1205 page 중에 216 page까지 완료 / 216
suare의 1205 page 중에 217 page까지 완료 / 217
suare의 1205 page 중에 218 page까지 완료 / 218
suare의 1205 page 중에 219 page까지 완료 / 219
suare의 1205 page 중에 220 page까지 완료 / 220
suare의 1205 page 중에 221 page까지 완료 / 221
suare의 1205 page 중에 222 page까지 완료 / 222
suare의 1205 page 중에 223 page까지 완료 / 223
suare의 1205 page 중에 224 page까지 완료 / 224
suare의 1205 page 중에 225 page까지 완료 / 225
suare의 1205 page 중에 226 page까지 완료 / 226
suare의 1205 page 중에 227 page까지 완료 / 227
suare의 1205 page 중에 228 page까지 완료 / 228
suare의 1205 page 중에 229 page까지 완료 / 229
suare의 1205 page 중에 230 page까지 완료 / 230
suare의 1205 page 중에 231 page까지 완료 / 231
suare의 1205 page 중에 232 page까지 완료 / 232
suare의 1205 page 중에 233 page까지 완료 / 233
suare의 1205 page 중에 234 page까지 완료 / 234
suare의 1205 page 중에 235 page까지 완료 / 235


suare의 1205 page 중에 415 page까지 완료 / 415
suare의 1205 page 중에 416 page까지 완료 / 416
suare의 1205 page 중에 417 page까지 완료 / 417
suare의 1205 page 중에 418 page까지 완료 / 418
suare의 1205 page 중에 419 page까지 완료 / 419
suare의 1205 page 중에 420 page까지 완료 / 420
suare의 1205 page 중에 421 page까지 완료 / 421
suare의 1205 page 중에 422 page까지 완료 / 422
suare의 1205 page 중에 423 page까지 완료 / 423
suare의 1205 page 중에 424 page까지 완료 / 424
suare의 1205 page 중에 425 page까지 완료 / 425
suare의 1205 page 중에 426 page까지 완료 / 426
suare의 1205 page 중에 427 page까지 완료 / 427
suare의 1205 page 중에 428 page까지 완료 / 428
suare의 1205 page 중에 429 page까지 완료 / 429
suare의 1205 page 중에 430 page까지 완료 / 430
suare의 1205 page 중에 431 page까지 완료 / 431
suare의 1205 page 중에 432 page까지 완료 / 432
suare의 1205 page 중에 433 page까지 완료 / 433
suare의 1205 page 중에 434 page까지 완료 / 434
suare의 1205 page 중에 435 page까지 완료 / 435
suare의 1205 page 중에 436 page까지 완료 / 436
suare의 1205 page 중에 437 page까지 완료 / 437
suare의 1205 page 중에 438 page까지 완료 / 438
suare의 1205 page 중에 439 page까지 완료 / 439


suare의 1205 page 중에 619 page까지 완료 / 619
suare의 1205 page 중에 620 page까지 완료 / 620
suare의 1205 page 중에 621 page까지 완료 / 621
suare의 1205 page 중에 622 page까지 완료 / 622
suare의 1205 page 중에 623 page까지 완료 / 623
suare의 1205 page 중에 624 page까지 완료 / 624
suare의 1205 page 중에 625 page까지 완료 / 625
suare의 1205 page 중에 626 page까지 완료 / 626
suare의 1205 page 중에 627 page까지 완료 / 627
suare의 1205 page 중에 628 page까지 완료 / 628
suare의 1205 page 중에 629 page까지 완료 / 629
suare의 1205 page 중에 630 page까지 완료 / 630
suare의 1205 page 중에 631 page까지 완료 / 631
suare의 1205 page 중에 632 page까지 완료 / 632
suare의 1205 page 중에 633 page까지 완료 / 633
suare의 1205 page 중에 634 page까지 완료 / 634
suare의 1205 page 중에 635 page까지 완료 / 635
suare의 1205 page 중에 636 page까지 완료 / 636
suare의 1205 page 중에 637 page까지 완료 / 637
suare의 1205 page 중에 638 page까지 완료 / 638
suare의 1205 page 중에 639 page까지 완료 / 639
suare의 1205 page 중에 640 page까지 완료 / 640
suare의 1205 page 중에 641 page까지 완료 / 641
suare의 1205 page 중에 642 page까지 완료 / 642
suare의 1205 page 중에 643 page까지 완료 / 643


suare의 1205 page 중에 823 page까지 완료 / 823
suare의 1205 page 중에 824 page까지 완료 / 824
suare의 1205 page 중에 825 page까지 완료 / 825
suare의 1205 page 중에 826 page까지 완료 / 826
suare의 1205 page 중에 827 page까지 완료 / 827
suare의 1205 page 중에 828 page까지 완료 / 828
suare의 1205 page 중에 829 page까지 완료 / 829
suare의 1205 page 중에 830 page까지 완료 / 830
suare의 1205 page 중에 831 page까지 완료 / 831
suare의 1205 page 중에 832 page까지 완료 / 832
suare의 1205 page 중에 833 page까지 완료 / 833
suare의 1205 page 중에 834 page까지 완료 / 834
suare의 1205 page 중에 835 page까지 완료 / 835
suare의 1205 page 중에 836 page까지 완료 / 836
suare의 1205 page 중에 837 page까지 완료 / 837
suare의 1205 page 중에 838 page까지 완료 / 838
suare의 1205 page 중에 839 page까지 완료 / 839
suare의 1205 page 중에 840 page까지 완료 / 840
suare의 1205 page 중에 841 page까지 완료 / 841
suare의 1205 page 중에 842 page까지 완료 / 842
suare의 1205 page 중에 843 page까지 완료 / 843
suare의 1205 page 중에 844 page까지 완료 / 844
suare의 1205 page 중에 845 page까지 완료 / 845
suare의 1205 page 중에 846 page까지 완료 / 846
suare의 1205 page 중에 847 page까지 완료 / 847


...

branded의 847 page 중에 45 page까지 완료 / 4215
branded의 847 page 중에 46 page까지 완료 / 4216
branded의 847 page 중에 47 page까지 완료 / 4217
branded의 847 page 중에 48 page까지 완료 / 4218
branded의 847 page 중에 49 page까지 완료 / 4219
branded의 847 page 중에 50 page까지 완료 / 4220
branded의 847 page 중에 51 page까지 완료 / 4221
branded의 847 page 중에 52 page까지 완료 / 4222
branded의 847 page 중에 53 page까지 완료 / 4223
branded의 847 page 중에 54 page까지 완료 / 4224
branded의 847 page 중에 55 page까지 완료 / 4225
branded의 847 page 중에 56 page까지 완료 / 4226
branded의 847 page 중에 57 page까지 완료 / 4227
branded의 847 page 중에 58 page까지 완료 / 4228
branded의 847 page 중에 59 page까지 완료 / 4229
branded의 847 page 중에 60 page까지 완료 / 4230
branded의 847 page 중에 61 page까지 완료 / 4231
branded의 847 page 중에 62 page까지 완료 / 4232
branded의 847 page 중에 63 page까지 완료 / 4233
branded의 847 page 중에 64 page까지 완료 / 4234
branded의 847 page 중에 65 page까지 완료 / 4235
branded의 847 page 중에 66 page까지 완료 / 4236
branded의 847 page 중에 67 page까지 완료 / 4237
branded의 847 page 중에 68 page까지 완료 / 4238
branded의 847 pag

branded의 847 page 중에 241 page까지 완료 / 4411
branded의 847 page 중에 242 page까지 완료 / 4412
branded의 847 page 중에 243 page까지 완료 / 4413
branded의 847 page 중에 244 page까지 완료 / 4414
branded의 847 page 중에 245 page까지 완료 / 4415
branded의 847 page 중에 246 page까지 완료 / 4416
branded의 847 page 중에 247 page까지 완료 / 4417
branded의 847 page 중에 248 page까지 완료 / 4418
branded의 847 page 중에 249 page까지 완료 / 4419
branded의 847 page 중에 250 page까지 완료 / 4420
branded의 847 page 중에 251 page까지 완료 / 4421
branded의 847 page 중에 252 page까지 완료 / 4422
branded의 847 page 중에 253 page까지 완료 / 4423
branded의 847 page 중에 254 page까지 완료 / 4424
branded의 847 page 중에 255 page까지 완료 / 4425
branded의 847 page 중에 256 page까지 완료 / 4426
branded의 847 page 중에 257 page까지 완료 / 4427
branded의 847 page 중에 258 page까지 완료 / 4428
branded의 847 page 중에 259 page까지 완료 / 4429
branded의 847 page 중에 260 page까지 완료 / 4430
branded의 847 page 중에 261 page까지 완료 / 4431
branded의 847 page 중에 262 page까지 완료 / 4432
branded의 847 page 중에 263 page까지 완료 / 4433
branded의 847 page 중에 264 page까지 완료

branded의 847 page 중에 436 page까지 완료 / 4606
branded의 847 page 중에 437 page까지 완료 / 4607
branded의 847 page 중에 438 page까지 완료 / 4608
branded의 847 page 중에 439 page까지 완료 / 4609
branded의 847 page 중에 440 page까지 완료 / 4610
branded의 847 page 중에 441 page까지 완료 / 4611
branded의 847 page 중에 442 page까지 완료 / 4612
branded의 847 page 중에 443 page까지 완료 / 4613
branded의 847 page 중에 444 page까지 완료 / 4614
branded의 847 page 중에 445 page까지 완료 / 4615
branded의 847 page 중에 446 page까지 완료 / 4616
branded의 847 page 중에 447 page까지 완료 / 4617
branded의 847 page 중에 448 page까지 완료 / 4618
branded의 847 page 중에 449 page까지 완료 / 4619
branded의 847 page 중에 450 page까지 완료 / 4620
branded의 847 page 중에 451 page까지 완료 / 4621
branded의 847 page 중에 452 page까지 완료 / 4622
branded의 847 page 중에 453 page까지 완료 / 4623
branded의 847 page 중에 454 page까지 완료 / 4624
branded의 847 page 중에 455 page까지 완료 / 4625
branded의 847 page 중에 456 page까지 완료 / 4626
branded의 847 page 중에 457 page까지 완료 / 4627
branded의 847 page 중에 458 page까지 완료 / 4628
branded의 847 page 중에 459 page까지 완료

review 저장완료_96000
branded의 847 page 중에 631 page까지 완료 / 4801
branded의 847 page 중에 632 page까지 완료 / 4802
branded의 847 page 중에 633 page까지 완료 / 4803
branded의 847 page 중에 634 page까지 완료 / 4804
branded의 847 page 중에 635 page까지 완료 / 4805
branded의 847 page 중에 636 page까지 완료 / 4806
branded의 847 page 중에 637 page까지 완료 / 4807
branded의 847 page 중에 638 page까지 완료 / 4808
branded의 847 page 중에 639 page까지 완료 / 4809
branded의 847 page 중에 640 page까지 완료 / 4810
branded의 847 page 중에 641 page까지 완료 / 4811
branded의 847 page 중에 642 page까지 완료 / 4812
branded의 847 page 중에 643 page까지 완료 / 4813
branded의 847 page 중에 644 page까지 완료 / 4814
branded의 847 page 중에 645 page까지 완료 / 4815
branded의 847 page 중에 646 page까지 완료 / 4816
branded의 847 page 중에 647 page까지 완료 / 4817
branded의 847 page 중에 648 page까지 완료 / 4818
branded의 847 page 중에 649 page까지 완료 / 4819
branded의 847 page 중에 650 page까지 완료 / 4820
branded의 847 page 중에 651 page까지 완료 / 4821
branded의 847 page 중에 652 page까지 완료 / 4822
branded의 847 page 중에 653 page까지 완료 / 4823
branded의 847 pag

branded의 847 page 중에 826 page까지 완료 / 4996
branded의 847 page 중에 827 page까지 완료 / 4997
branded의 847 page 중에 828 page까지 완료 / 4998
branded의 847 page 중에 829 page까지 완료 / 4999
branded의 847 page 중에 830 page까지 완료 / 5000
review 저장완료_100000
branded의 847 page 중에 831 page까지 완료 / 5001
branded의 847 page 중에 832 page까지 완료 / 5002
branded의 847 page 중에 833 page까지 완료 / 5003
branded의 847 page 중에 834 page까지 완료 / 5004
branded의 847 page 중에 835 page까지 완료 / 5005
branded의 847 page 중에 836 page까지 완료 / 5006
branded의 847 page 중에 837 page까지 완료 / 5007
branded의 847 page 중에 838 page까지 완료 / 5008
branded의 847 page 중에 839 page까지 완료 / 5009
branded의 847 page 중에 840 page까지 완료 / 5010
branded의 847 page 중에 841 page까지 완료 / 5011
branded의 847 page 중에 842 page까지 완료 / 5012
branded의 847 page 중에 843 page까지 완료 / 5013
branded의 847 page 중에 844 page까지 완료 / 5014
branded의 847 page 중에 845 page까지 완료 / 5015
branded의 847 page 중에 846 page까지 완료 / 5016
branded의 847 page 중에 847 page까지 완료 / 5017
nomanual의 519 page 중에 1 page까지 완료 / 5018
nomanual의 519 pa

In [6]:
review_df.to_csv(f"review/review_{count*20}.csv")
print(f"review 저장완료_{count*20}")

review 저장완료_103300


In [451]:
url = f'https://www.musinsa.com/goods/reviews/lists?type=style&searchYear=2022&searchMonth=&searchDay=&maxRt=2022&minRt=2009&brand={brand}&page=1&sort=comment_new_desc'
driver.get(url)
soup = BeautifulSoup(driver.page_source,"html.parser")

In [14]:
brand = pd.read_excel("brand_name_real.xlsx")
brand.drop(columns=['Unnamed: 0','Unnamed: 2'],inplace=True)
brand_list = [text.replace(' ', '').lower() for text in list(brand.loc[brand['나머지']==3,0])]

In [15]:
fail_brand

['nomanual',
 'niceghostclub',
 'cpgnstudio',
 'pottery',
 'filluminate',
 'thomasmore',
 'fallett',
 'mmlg',
 'puma',
 'mahagrid',
 'anotheroffice',
 'millo',
 'thrasher',
 'lmood',
 'ganni',
 'gainsboro',
 'lmc',
 'archivebold',
 'waikei',
 'placestudio',
 'camelwork',
 'nerdy',
 'dynafit',
 'noirer',
 'underarmour',
 'lordjohngrey',
 'anewgolf',
 'dnsr',
 'critic',
 'guess',
 'pianer',
 'laugher',
 'venhit',
 'asifcalie',
 '247series',
 'markgonzales',
 'diamondlayla',
 'piecemaker',
 'heim',
 'ordinaryholiday',
 'dunstforwomen',
 'dearme',
 'gammagallery',
 'knowwave',
 'bsrabbit',
 'ojos',
 'namerclothing',
 '999humanity',
 '1989standard',
 'notyourrose',
 'chasecult',
 'inspector',
 'letterfrommoon',
 'jipsy',
 'ninetyplus',
 'ironypornowhiteline',
 'pinggolf',
 'shirter',
 'lookast',
 'maisonkitsune',
 'spadeclubseoul',
 'badblue',
 'tbj',
 'generalidea',
 'yuppe',
 'kolonsport',
 'frei',
 'goyowear',
 'mucent',
 'zeroplanet',
 'dazedayz',
 'bibyseob',
 'toez',
 'undyedroom',
 '

In [6]:
tot_page = 14000
page = 4000
brand = "musinsa"
print(f"{brand}의 {1} page 중에 {2} page까지 완료")

musinsa의 1 page 중에 2 page까지 완료


In [14]:
brand_list[101]

'amomento'

In [418]:
soup.find_all("div",{"class":"review-list"})[18].find_all('img')[2]

<img alt="무신사 스탠다드(MUSINSA STANDARD) 리얼 와이드 히든 밴딩 슬랙스 [블랙] 후기" src="//image.msscdn.net/data/estimate/1168922_0/gallery_647c3443cc523.jpg.view"/>

In [369]:
brand = pd.read_excel("brand_name.xlsx")
brand_list = [text.replace(' ', '').lower() for text in list(brand[0])]
brand_list

['musinsastandard',
 'matinkim',
 'adidas',
 'suare',
 'yale',
 'glw',
 'dimitriblack',
 'lafudgestore',
 'toffee',
 'travel',
 'covernat',
 'drawfit',
 'satur',
 'avandress',
 'thenorthface',
 'xero',
 'spao',
 'codegraphy',
 'poloralphlauren',
 'newbalance',
 'calvinkleinjeans',
 'nationalgeographic',
 'espionage',
 'anoblir',
 'outstanding',
 'trillion',
 'umbro',
 'lee',
 'they',
 'marithefrancoisgirbaud',
 'thisisneverthat',
 'fp142',
 'partimento',
 'physicaleducationdepartment',
 'jemut',
 'mlb',
 'sculptor',
 'nodearchive',
 'mardimercredi',
 'branded',
 'brownbreath',
 'perstep',
 'not4nerd',
 'minav',
 'uniformbridge',
 'nomanual',
 'whatitisnt',
 'intempomood',
 'soverment',
 'visionstreetwear',
 'discoveryexpedition',
 'niceghostclub',
 'anderssonbell',
 'xtonz',
 'fottners',
 'maisonmined',
 'takeasy',
 'cpgnstudio',
 'signature',
 'giordano',
 'asics',
 'coor',
 'fila',
 'pottery',
 'partimentowomen',
 'kodak',
 'gakkaiunions',
 'brenson',
 'juanhomme',
 'filluminate',
 '

In [394]:
path = r"C:\Users\NT550\Desktop\chromedriver.exe" # 크롬드라이버 경로 입력
s = Service(path)
chrome_options = Options()
chrome_options.add_argument('--headless')  # 화면에 브라우저가 나타나지 않도록 headless 모드로 실행합니다.

driver = webdriver.Chrome(service=s)
brand = 'musinsastandard'
url = f'https://www.musinsa.com/goods/reviews/lists?type=style&searchYear=2023&searchMonth=&searchDay=&maxRt=2023&minRt=2009&brand={brand}&page=1&sort=new&hashId=&bestType=&s_type=all&searchKeyword='
driver.get(url)
soup = BeautifulSoup(driver.page_source,"html.parser")

In [381]:
tot_page = re.sub(r'[^\d]', '', soup.find_all("span",{"class":"totalPagingNum"})[0].text)
int(tot_page)

6124

In [326]:
for i in range(20):
        # goods_id
        url = soup.find_all("div",{"class":"review-list"})[i].find_all('a',{"class":"review-goods-information__link"})[0].get('href')
        match = re.search(r'goods/(\d+)/', url)
        if match:
            goods_id = match.group(1)

        # user_id
        user_id = soup.find_all("div",{"class":"review-list"})[i].find_all("p")[0].text
        
        # date
        date = soup.find_all("div",{"class":"review-list"})[i].find_all("p")[1].text
        
        # user 정보 ('여성 ', ' 160cm ', ' 52kg') / 등록되어 있지 않은 경우도 있음
        if soup.find_all("div",{"class":"review-list"})[i].find_all("p",{"class":"review-profile__body_information"}) != []:
            user_sex, user_height, user_weight = soup.find_all("div",{"class":"review-list"})[3].find_all("p",{"class":"review-profile__body_information"})[0].text.split("·")
        
        # Size 
        size = re.sub(r'\s+', '', soup.find_all("div",{"class":"review-list"})[i].find_all("span",{"class":"review-goods-information__option"})[0].text)
        
        # url
        goods_url = soup.find_all("div",{"class":"review-list"})[i].find_all('a')[2].get('href')
        #brand
        goods_brand = soup.find_all("div",{"class":"review-list"})[i].find_all('a')[3].text
        # goods name
        goods_name = soup.find_all("div",{"class":"review-list"})[i].find_all('a')[4].text
        
        # 도움돼요 /스타일 좋아요 / 평가
        help_eval = soup.find_all("div",{"class":"review-list"})[i].find_all('li',{"class":"review-evaluation-button__list"})[0].find('span').text
        style_eval = soup.find_all("div",{"class":"review-list"})[i].find_all('li',{"class":"review-evaluation-button__list"})[1].find('span').text
        eval_list = [item.get_text(strip=True) for item in soup.find_all("div",{"class":"review-list"})[i].find_all('li',{"class":"review-evaluation__item"})]
        rating = re.find(r'\d+', soup.find_all("div",{"class":"review-list"})[i].find_all('span',{"class":"review-list__rating__active"})[0].get("style"))[0]
        print(goods_id,user_id,date,user_sex, user_height, user_weight,size,goods_url,goods_brand,goods_name,help_eval,style_eval,eval_list,rating)

1323612 LV.4내사랑뽀로로 15일 전 남성   177cm   68kg BLACK/FREE https://www.musinsa.com/app/goods/1323612/0 앤더슨벨 유니섹스 테크니컬 베를린 백팩 aaa237u 0 0 ['사이즈보통이에요', '밝기보통이에요', '색감보통이에요', '수납공간적당해요', '무게감적당해요', '배송빨라요', '포장꼼꼼해요'] 100
1854665 LV.5yesoozn 2시간 전 남성   177cm   68kg L https://www.musinsa.com/app/goods/1854665/0 엑스트라오디너리 UTILITY RIPSTOP PANTS BEIGE 0 0 ['사이즈보통이에요', '밝기보통이에요', '색감보통이에요', '두께감보통이에요', '배송빨라요', '포장꼼꼼해요'] 100
3054118 LV.5Solll1 21일 전 남성   177cm   68kg FREE https://www.musinsa.com/app/goods/3054118/0 본챔스 BCG 버튼 니트 볼레로 가디건 BG22FT17IV 0 0 ['사이즈커요', '밝기보통이에요', '색감선명해요', '두께감보통이에요', '촉감부드러워요', '배송빨라요', '포장꼼꼼해요'] 100
1931720 LV.5알게뭐야! 29일 전 남성   177cm   68kg L https://www.musinsa.com/app/goods/1931720/0 라퍼지스토어 [23SS Ver.] 클래식테이퍼드 와이드 원턱 린넨 팬츠_Oatmeal 1 0 ['사이즈보통이에요', '밝기보통이에요', '색감보통이에요', '두께감보통이에요', '배송아쉬워요', '포장꼼꼼해요'] 100
3118912 LV.3뉴비_a6af3b84 2023.05.03 남성   177cm   68kg S https://www.musinsa.com/app/goods/3118912/0 스컬프터 Sports Jersey Tee Pink 0 2 ['사이즈작아요', '밝기밝아요', '색감보통이에요', '두께감얇아요

In [117]:
# user_id
user_id = soup.find_all("div",{"class":"review-list"})[0].find_all("p")[0].text

# date
date = soup.find_all("div",{"class":"review-list"})[0].find_all("p")[1].text
user_id,date

# user info
user_sex, user_height, user_weight = soup.find_all("div",{"class":"review-list"})[0].find_all("p")[2].text.split('·')

# url
goods_url = soup.find_all("div",{"class":"review-list"})[0].find_all('a')[2].get('href')
#brand
brand = soup.find_all("div",{"class":"review-list"})[0].find_all('a')[3].text

('LV.3뉴비_a6af3b84', '2023.05.03')